In [144]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Aim is to predict the marks of students of the test data

In [145]:
# Use the file namd 'training data' to train the model

data = pd.read_excel('Training data.xlsx')
x_train = np.array(data.iloc[:,0:8])
y_train = np.array(data.iloc[:,8]).reshape(-1,1)
# print(len(x_train[:,0]))
# print()


# To get an idea whether to add some features or not
# I am choosing those features which are giving a different range of marks on
# its different feature_vlalue- internet, studytime, freetime(not much contri)
# absences and iq

# plt.plot(x_train[:,7], y_train, '.', markersize=2)


# Also do label encoding for features not represented in numbers
# refer the link if not know : https://youtu.be/589nCGeWG1w?si=t2Wa7LgbUOO4RooM


def feature_changing(x_train):
  for i in range(x_train.shape[0]):
    if(x_train[i,0] == 'no'):
      x_train[i,0] = 0
    else:
      x_train[i,0] = 1
  # x_train = np.delete(x_train,[1,2,6],axis=1)

  for i in range(x_train.shape[0]):
    if(x_train[i,1] == 'M'):
      x_train[i,1] = 1
    else:
      x_train[i,1] = 0

  for i in range(x_train.shape[0]):
      x_train[i,6] = x_train[i,3]/x_train[i,4]

  # x_train[:,1] = x_train[:,1]/4
  # x_train[:,2] = x_train[:,1]/5
  # x_train[:,3] = x_train[:,1]/9
  # x_train[:,4] = x_train[:,1]/129
  return x_train

x_train = feature_changing(x_train)

In [146]:
def z_score(x_train):

  x_std = np.std(x_train, axis=0)
  x_mean = np.mean(x_train, axis =0)
  x_std[x_std == 0] = 1e-10  # Replace zeros with a small number to avoid division by zero
  x_train = (x_train - x_mean) / x_std

  return x_train,x_std,x_mean

In [147]:
def cost(x_train,y_train,w,b):
  m = x_train.shape[0]
  loss = 0.0
  # f_wb_i = np.zeros(1001, dtype = 'float64')
  for i in range(0,999):
    f_wb_i = np.dot(x_train[i], w) + b
    loss = loss + (f_wb_i - y_train[i])**2
    # f_wb_i[i] = (f_wb_i[i] - y_train[i])**2
  # loss = np.sum(f_wb_i)
  loss = loss / (2 * m)

  return loss

# w_init = np.ones(4)
# b_init = 0.0
# c = cost(x_train, y_train, w_init, b_init)
# print(c)
# print()

In [148]:
def compute_gradient(X, y, w, b):
    """
    Computes the gradient for linear regression
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters
      b (scalar)       : model parameter

    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w.
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b.
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros(n)
    dj_db = 0.

    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err * X[i, j]
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_db, dj_dw

In [149]:
def gradient_descent(x_train,y_train,w_in,b_in):

    w = w_in  #avoid modifying global w within function
    b = b_in

    for i in range(1000):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = compute_gradient(x_train, y_train, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - 0.01 * dj_dw               ##None
        b = b - 0.01 * dj_db               ##None

    return w,b

In [150]:
x_train = x_train.astype(np.float64)
x_train,x_std,x_mean = z_score(x_train)

np.random.seed(2147483647)
w = np.random.randn(x_train.shape[1])
b = np.random.randn(1)

old_cost = 0

while abs(old_cost - cost(x_train,y_train,w,b))>0.00001:
  old_cost = cost(x_train,y_train,w,b)
  w,b = gradient_descent(x_train,y_train,w,b)

x_predict = pd.read_excel('Test data.xlsx').iloc[:,:8].to_numpy()
x_predict = feature_changing(x_predict)
x_predict = (x_predict - x_mean)/x_std
ans = pd.read_excel('Test data.xlsx').iloc[:,8].to_numpy()

y_predict = np.dot(x_predict,w) + b

accuracy = 0
for dim in range(len(ans)):
  if abs(y_predict[dim]-ans[dim])<0.5: # do not change the tolerance as you'll be checked on +- 0.5 error only
    accuracy += 1
accuracy = round(accuracy*100/200.0,2)
ok = 'Congratulations' if accuracy>95 else 'Optimization required'
print(f"{ok}, your accuracy is {accuracy}%")

<ipython-input-148-4dd75af69efc>:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dj_dw[j] = dj_dw[j] + err * X[i, j]


Congratulations, your accuracy is 100.0%
